In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

class GoogleNewsScraper():
    def __init__(self, query, start_url=None):
        self.query = query
        self.BASE_URL = 'https://www.google.com/search?q=%s&tbm=nws'
        if start_url is None:
            self.url = self.BASE_URL % self.query
        else:
            self.url = start_url
        
        self.raw_links = []
        print('start url: %s' % self.url)

    def start_browser(self):
        self.browser = webdriver.Firefox()
        self.browser.get(self.url)
        print('current browser location: %s' % self.browser.current_url)

    def scrape_links(self):
        link_elements = self.browser.find_elements_by_tag_name('a')
        links_raw = list(map(lambda ele : ele.get_attribute('href'), link_elements))
        links_raw = list(filter(lambda l : l is not None, links_raw))
        return links_raw
    
    def go_next(self):
        try:
            self.browser.find_element_by_id('pnnext').click()
        except NoSuchElementException:
            return False
        return True
    
    def scrape_loop(self, verbose=True, skip_on_error=True):
        try:
            self.last_url = self.browser.current_url
            if verbose:
                print('Scraping from %s...' % self.last_url)

            new_links = self.scrape_links()
            self.raw_links.extend(new_links)

            if verbose:
                print('Found %d new links' % len(new_links))
                print('Total links: %d' % len(self.raw_links))
        except Exception as e:
            if skip_on_error:
                print('Exception: ', e)
            else:
                raise e
        
        return self.go_next()

    def stop_browser(self):
        self.browser.quit()

In [67]:
from urllib.parse import quote

query = 'coronavirus world'
query = quote(query)

In [68]:
scraper = GoogleNewsScraper(query)

start url: https://www.google.com/search?q=coronavirus%20world&tbm=nws


In [69]:
scraper.start_browser()

current browser location: https://www.google.com/search?q=coronavirus%20world&tbm=nws


In [70]:
import csv
import os

def save_csv(urls, file_path):
    with open(file_path, 'w') as file_out:
        writer = csv.writer(file_out, quoting=csv.QUOTE_ALL)
        writer.writerow(['url'])
        for link in urls:
            writer.writerow([link])

checkpoint_folder = 'scraped/' + query

os.mkdir(checkpoint_folder)

counter = 0
while scraper.scrape_loop(verbose=False):
    if counter % 10 == 0:
        checkpoint_path = checkpoint_folder + '/checkpoint_%d.csv' % len(scraper.raw_links)
        print('saving checkpoint at %s...' % checkpoint_path)
        save_csv(scraper.raw_links, checkpoint_path)
    counter += 1


print('finished with %d links, saving final csv...' % len(scraper.raw_links))
save_csv(scraper.raw_links, checkpoint_folder + '/FINAL.csv')

print('stopping browser...')
scraper.stop_browser()
!taskkill /f /im geckodriver.exe

saving checkpoint at scraped/coronavirus%20world/checkpoint_60.csv...
saving checkpoint at scraped/coronavirus%20world/checkpoint_678.csv...
saving checkpoint at scraped/coronavirus%20world/checkpoint_1308.csv...
saving checkpoint at scraped/coronavirus%20world/checkpoint_1945.csv...
finished with 2170 links, saving final csv...
stopping browser...


ERROR: The process "geckodriver.exe" not found.


In [49]:
scraper.stop_browser()

In [50]:
!taskkill /f /im geckodriver.exe

ERROR: The process "geckodriver.exe" not found.
